In [ ]:
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt
import pint
ureg = pint.UnitRegistry()

class Link:

    empty = ()

    def __init__(self, first, rest=empty):
        assert rest is Link.empty or isinstance(
            rest, Link), "Link does not follow proper structure"
        self.first = first
        self.rest = rest

    def __repr__(self):
        if self.rest is not Link.empty:
            rest_repr = ', ' + repr(self.rest)
        else:
            rest_repr = ''
        return 'Link(' + repr(self.first) + rest_repr + ')'

    def __str__(self):
        string = '<'
        while self.rest is not Link.empty:
            string += str(self.first) + ' '
            self = self.rest
        return string + str(self.first) + '>'

class System:

    '''
    self.delta_attribute = [inital_value, end_value]
    example = System(attribute:unit, value)
    '''

    def __init__(self, *argv):
        for i in range(len(argv)):
            if i % 2 == 0: # if is is even, including the first index
                if ':' in argv[i]:
                    temp = argv[i].split(':')

                    if 'delta_' in temp[0]:
                        setattr(self, temp[0], [argv[i+1][0] * ureg(temp[1]), argv[i+1][1] * ureg(temp[1])])
                    else:
                        setattr(self, temp[0], argv[i+1] * ureg(temp[1]))
                else:
                    setattr(self, argv[i], argv[i+1])

    def __str__(self):
        values = ''
        for attribute, value in sorted(vars(self).items()):
            if 'delta_' in attribute:
                values += f'\ninitial_{attribute.replace('delta_', '')} = {value[0]}'
                values += f'\nfinal_{attribute.replace('delta_', '')} = {value[1]}'
            else:
                values+= f'\n{attribute} = {value}'

        # add so it can print the delta attributes
        return values.replace('_', ' ') + '\n'

    def set_default_values(self, attributes, values):
        '''
        Attributes is a list of attribute names, values is the default values.
        The function will set those attributes if they are not defined already to those defaults in values.
        '''
        for i in range(len(attributes)):
            if ':' in attributes[i]:
                attributes[i] = attributes[i].split(':')
                if not hasattr(self, attributes[i][0]):
                    setattr(self, attributes[i][0], values[i] * ureg(attributes[i][1]))
            else:
                if not hasattr(self, attributes[i]):
                    setattr(self, attributes[i], values[i])
        return

    def has_values(self,attributes):
        '''
        Returns
        -------
        Raises an error if an attribute is not defined.
        '''
        for i in attributes:
            if not hasattr(self, i):
                raise AttributeError(f'{i} must be defined for {self.name if hasattr(self, 'name') else 'unnamed object'}.')

# Functions above this are for ease of use, error checking, indexing stuff, etc...
########################################################
# Functions below are finders. Some of them return things.

    def reynolds_number_finder(self):
        '''
        Will set self.reynolds_number to the result but will not return anything.
        '''
        self.has_values(['velocity', 'fluid_density', 'fluid_viscosity', 'pipe_diameter'])

        @ureg.wraps(None, ('kg/m**3', 'm/s', 'm', 'Pa * s'))
        def reynolds_number_function(fluid_density, velocity, pipe_diameter, fluid_viscosity):
            return fluid_density * velocity * pipe_diameter / fluid_viscosity

        setattr(self, 'reynolds_number',
                reynolds_number_function(self.fluid_density, self.velocity, self.pipe_diameter, self.fluid_viscosity))
        return

    def average_velocity_finder(self, units = 'm/s'):
        '''
        Will set self.velocity to the result but will not return anything.
        '''
        self.has_values(['volumetric_flow_rate', 'pipe_diameter'])

        @ureg.wraps(units, ('m**3/s', 'm'))
        def average_velocity_function(volumetric_flow_rate, pipe_diameter):
            return volumetric_flow_rate / (3.1415/4 * pipe_diameter**2)

        setattr(self, 'velocity', average_velocity_function(self.volumetric_flow_rate, self.pipe_diameter))
        return

    def friction_factor_using_haaland_finder(self):
        '''
        Will set self.friction_factor to the result but will not return anything.
        If Reynolds Number is < 4000, will assume laminar.
        '''
        self.has_values(['reynolds_number', 'pipe_diameter'])
        self.set_default_values(['epsilon'], [0])

        if self.reynolds_number < 4000:
            setattr(self, 'friction_factor', 64 / self.reynolds_number)
            return

        @ureg.wraps(None, (None, 'm', 'm'))
        def friction_factor_haaland_function(reynolds_number, epsilon, pipe_diameter):
            return (1/(-1.8 * np.log10(6.9 / (reynolds_number, +
                                           epsilon / pipe_diameter / 3.7 )**1.11)))**0.5

        setattr(self, 'friction_factor', friction_factor_haaland_function(self.reynolds_number, self.epsilon, self.pipe_diameter)
                )
        return

    def friction_factor_using_colebrook_finder(self):
        '''
        Will set self.friction_factor to the result but will not return anything.
        If Reynolds Number is < 4000, will assume laminar.
        '''
        self.has_values(['pipe_diameter', 'reynolds_number'])
        self.set_default_values(['epsilon'], [0])

        if self.reynolds_number < 2300:
            setattr(self, 'friction_factor', 64 / self.reynolds_number)
            return

        def residual(friction_factor):
            left_side = 1 / friction_factor**0.5

            @ureg.wraps(None, ('m', 'm', None))
            def right_side(epsilon, pipe_diameter, reynolds_number):
                return -2 * np.log10(epsilon / pipe_diameter / 3.7 + 2.51 / reynolds_number / friction_factor ** 0.5)

            return left_side - right_side(self.epsilon, self.pipe_diameter, self.reynolds_number)

        setattr(self, 'friction_factor', opt.fsolve(residual, 0.001)[0])
        return

    def head_loss_finder(self, units = 'm'):
        '''
        Will set self.head_loss to the result but will not return anything.
        '''
        self.has_values(['pipe_diameter', 'pipe_length', 'friction_factor', 'velocity'])
        self.set_default_values(['loss_coefficients'], [0])

        @ureg.wraps(units, (None, 'm', 'm', None, 'm/s'))
        def head_loss_function(friction_factor, pipe_length, pipe_diameter, loss_coefficients, velocity):
            g = 9.8

            return (friction_factor * pipe_length / pipe_diameter +
                    (sum(loss_coefficients) if isinstance(loss_coefficients, list) else loss_coefficients)) * velocity**2 / 2 / g

        setattr(self, 'head_loss', head_loss_function(self.friction_factor, self.pipe_length, self.pipe_diameter, self.loss_coefficients, self.velocity))
        return

    def pressure_loss_finder(self, units = 'Pa'):
        '''
        Will set self.pressure_loss to the result but will not return anything.
        '''
        self.has_values(['pipe_diameter', 'pipe_length', 'friction_factor', 'velocity'])
        self.set_default_values(['loss_coefficients'], [0])
        g = 9.8

        @ureg.wraps(units, ('kg/m**3', None, 'm', 'm', None, 'm/s'))
        def pressure_loss_function(fluid_density, friction_factor, pipe_length, pipe_diameter, velocity, loss_coefficients = 0):
            return fluid_density * (friction_factor * pipe_length / pipe_diameter +
                 (sum(loss_coefficients) if isinstance(loss_coefficients, list) else loss_coefficients)) * velocity**2 / 2

        setattr(self, 'pressure_loss',
                pressure_loss_function(self.fluid_density, self.friction_factor, self.pipe_length, self.pipe_diameter, self.velocity, self.loss_coefficients))
        return

    def non_newtonian_friction_factor_finder(self):
        '''
        Will set self.friction_factor to the result but will not return anything.
        If Reynolds Number is < 4000, will assume laminar.
        '''
        self.has_values(['reynolds_number', 'n'])

        def non_newtonian_friction_factor_function(reynolds_number, n):
            if reynolds_number < 4000:
                return 64 / reynolds_number
            else:
                c = 2**(n + 4) / 7**(7*n) * (4 * n / (3 * n + 1))**(3 * n**2)
                return 4 * (c / reynolds_number)**(1 / (3 * self.n + 1))

        setattr(self, 'friction_factor', non_newtonian_friction_factor_function(self.reynolds_number, self.n))
        return

    def non_newtonian_reynolds_number_finder(self):
        '''
        Will set self.reynolds_number to the result but will not return anything.
        '''
        self.has_values(['fluid_density', 'velocity', 'n', 'pipe_diameter'])
        self.set_default_values(['k'], [1.4]) # for air

        @ureg.wraps(None, ('kg/m**3', 'm/s', 'm', None, None))
        def non_newtonian_reynolds_number_function(fluid_density, velocity, pipe_diameter, n, k = 1.4):
            return 8 * fluid_density * velocity**(2 - self.n) * pipe_diameter ** n / (k * (2 * (3 * n + 1) / n)**n)

        setattr(self, 'reynolds_number', non_newtonian_reynolds_number_function(self.fluid_density, self.velocity, self.pipe_diameter, self.n, self.k))
        return

    def apparent_viscocity_finder(self, units = 'Pa.s'):
        '''
        Will set self.fluid_viscosity to the result but will not return anything.
        '''
        self.has_values(['shear_rate', 'n'])
        self.set_default_values(['k'],
                                [1.4]) # for air

        def apparent_viscosity_function(shear_rate, k, n):
            return k * shear_rate ** (n - 1)

        setattr(self, 'fluid_viscocity', apparent_viscosity_function(self.shear_rate, self.k, self.n) * ureg(units))
        return

    def choked_mass_flow_finder(self, units = 'kg/s'):
        '''
        This function will set self.mass_flow_rate assuming the flow is choked.
        '''
        self.has_values(['molecular_weight', 'orifice_diameter', 'tank_pressure',
                         'outside_pressure', 'tank_temperature'])
        self.set_default_values(['discharge_coefficient', 'k'], [1, 1.4])

        @ureg.wraps(units, ('Pa', 'K', 'kg/mol', None, None))
        def choked_mass_flow_function(tank_pressure, tank_temperature, molecular_weight, discharge_coefficient = 1, k = 1.4):
            gas_constant = 8.314
            area = 3.1415 / 4 * self.orifice_diameter**2
            return discharge_coefficient * area * tank_pressure * (k / gas_constant * molecular_weight /tank_temperature) ** 0.5 * (
                (2 / (k + 1)) ** ((k + 1) / 2 / (k  - 1)))

        setattr(self, 'mass_flow_rate', choked_mass_flow_function(self.tank_ressure, self.tank_temperature, self.molecular_weight, self.discharge_coefficient, self.k))
        return

    def mach_number_relief_sizing(self):
        '''
        Will set self.mach_number to the result. This is for 311. Flow must be choked.
        '''
        self.has_values(['tank_pressure', 'outside_pressure'])
        self.set_default_values(['k'], [1.4])

        @ureg.wraps(None, ('Pa', 'Pa', None))
        def mach_number_function(tank_pressure, outside_pressure, k = 1.4):
            mach = (2 / (k - 1) * ((tank_pressure / outside_pressure)**((k - 1)/k) - 1))
            return mach if mach < 1 else 1

        setattr(self, 'mach_number', mach_number_function(self.tank_pressure, self.outside_pressure, self.k))
        return

    def choked_flow_query(self):
        '''
        Returns True if flow is choked, otherwise returns False.
        '''
        self.has_values(['tank_pressure', 'outside_pressure'])
        self.set_default_values(['k'], [1.4])

        @ureg.wraps(None, ('Pa', 'Pa', None))
        def choked_flow_query_function(tank_pressure, outside_pressure, k = 1.4):
            return tank_pressure >= outside_pressure * ((k + 1) / 2) ** (k / (k - 1))

        return choked_flow_query_function(self.tank_pressure, self.outside, self.k)

    def speed_of_sound_finder(self, units = 'm/s'):
        '''
        Sets the self.speed_of_sound to the result but does not return anything.
        '''
        self.has_values(['molecular_weight', 'temperature'])
        self.set_default_values(['k'], [1.4])

        @ureg.wraps(units, ('K', 'kg.mol', None))
        def speed_of_sound_function(temperature, molecular_weight, k = 1.4):
            gas_constant = 8.314
            return (k * gas_constant * temperature / molecular_weight)**0.5

        setattr(self, 'speed_of_sound', )
        return

    def choked_flow_tao_finder(self, units):
        '''
        Sets self.choked_flow_tao to the result but does not return anything.
        '''
        self.has_values(['tank_volume', 'orifice_diameter', 'speed_of_sound'])
        self.set_default_values(['discharge_coefficient', 'k'], [1, 1.4])

        @ureg.wraps(units, ('m', 'm**3', None, 'm/s'))
        def choked_flow_tao_function(orifice_diameter, tank_volume, speed_of_sound, discharge_coefficient = 1, k = 1.4):
            area = 3.1415 / 4 * orifice_diameter**2
            return tank_volume / discharge_coefficient / area / speed_of_sound * ((k + 1) / 2)**((k + 1) / 2 / (k - 1))

        setattr(self, 'choked_flow_tao', choked_flow_tao_function(self.orifice_diameter, self.tank_volume, self.speed_of_sound, self.discharge_coefficient, self.k))
        return

    def choked_flow_adiabatic_tank_pressure_drop_finder(self, time, units = 'Pa'):
        '''
        Returns pressure of the tank given a time, assumes self.tank_pressure is the initial pressure.
        '''
        self.has_values(['tank_pressure', 'choked_flow_tao'])
        self.set_default_values(['discharge_coefficient', 'k'], [1, 1.4])

        @ureg.wraps(units, ('s', 'Pa', 's', None))
        def choked_flow_adiabatic_tank_pressure_drop_function(time, tank_pressure, choked_flow_tao, k = 1.4):
            return tank_pressure * (1 + (k -1) / 2 * time / choked_flow_tao)**(2 * k / (1 - k))

        return choked_flow_adiabatic_tank_pressure_drop_function(time, self.tank_pressure, self.choked_flow_tao, self.k)

    def choked_flow_isothermal_tank_pressure_drop_finder(self, time, units = 'Pa'):
        '''
        Returns pressure of the tank given a time, assumes self.tank_pressure is the initial pressure.
        '''
        self.has_values(['tank_pressure', 'choked_flow_tao'])
        self.set_default_values(['discharge_coefficient', 'k'], [1, 1.4])

        @ureg.wraps(units, ('Pa', 's', 's'))
        def choked_flow_isothermal_tank_pressure_drop_function(tank_pressure, time, choked_flow_tao):
            return tank_pressure * np.exp(-time / choked_flow_tao)

        return choked_flow_isothermal_tank_pressure_drop_function(self.tank_pressure, time, self.choked_flow_tao)

    def volumetric_flow_rate_from_pitot_tube(self, units = 'm**3/s'):
        '''
        Sets self.volumetric_flow_rate to the resulte but does not return anything.
        '''
        self.has_values(['delta_pressure', 'fluid_density', 'pipe_diameter'])

        @ureg.wraps(units, ('m', 'Pa', 'Pa', 'kg/m**3'))
        def volumetric_flow_from_pitot_tube_function(pipe_diameter, initial_pressure, final_pressure, fluid_density):
            area = 3.1415 / 4 * pipe_diameter**2
            velocity = (2 * (final_pressure - initial_pressure[1]) / fluid_density)**0.5
            return area * velocity

        setattr(self, 'volumetric_flow_rate',
                volumetric_flow_from_pitot_tube_function(self.pipe_diameter, self.delta_pressure[0], self.delta_pressure[1], self.fluid_density))
        return

    def volumetric_flow_rate_from_orifice_meter(self, units = 'm**3/s'):
        '''
        Sets self.volumetric_flow_rate to the result but doesn't return anything. Lowkey might not work since I changed it.
        '''
        self.has_values(['pipe_diameter', 'orifice_diameter', 'delta_pressure', 'fluid_density'])
        self.set_default_values(['expansion_factor'], [1])

        @ureg.wraps(units, ('m', 'm', 'Pa', 'Pa', 'kg/m**3', 'Pa.s', None))
        def volumetric_flow_rate_from_orifice_meter_function(
                orifice_diameter, pipe_diameter, initial_pressure, final_pressure, fluid_density, fluid_viscosity, expansion_factor = 1):
            beta = orifice_diameter / pipe_diameter
            area = 3.1415 / 4 * orifice_diameter ** 2

            def volumetric_flow_rate_function(discharge_coefficient, expansion_factor, initial_pressure, final_pressure, fluid_density):
                return expansion_factor * discharge_coefficient * area * (2 * (final_pressure - initial_pressure) / fluid_density / (1 - beta)**4)**0.5

            def discharge_coefficient_function(reynolds_number):
                return 0.5959 + 0.0312 * beta**1.1 - 0.184 * beta**8 + 91.78 * beta**2.5 / reynolds_number**0.75 if reynolds_number < 30000 else 0.61

            def average_velocity_function(volumetric_flow_rate, pipe_diameter):
                return volumetric_flow_rate / (3.1415/4 * pipe_diameter**2)

            def reynolds_number_function(fluid_density, velocity, pipe_diameter, fluid_viscosity):
                return fluid_density * velocity * pipe_diameter / fluid_viscosity

            volumetric_flow_rate = volumetric_flow_rate_function(0.61, expansion_factor, initial_pressure, final_pressure, fluid_density) # always guess 0.61
            velocity = average_velocity_function(volumetric_flow_rate, pipe_diameter)
            reynolds_number = reynolds_number_function(fluid_density, velocity, pipe_diameter, fluid_viscosity)

            while abs(discharge_coefficient_function(reynolds_number) - discharge_coefficient) > 0.001:
                discharge_coefficient =  discharge_coefficient_function(reynolds_number)
                volumetric_flow_rate = volumetric_flow_rate_function(discharge_coefficient, expansion_factor, initial_pressure, final_pressure, fluid_density)
                velocity = average_velocity_function(volumetric_flow_rate, pipe_diameter)
                reynolds_number = reynolds_number_function(fluid_density, velocity, pipe_diameter, fluid_viscosity)
            return volumetric_flow_rate

        return volumetric_flow_rate_from_orifice_meter_function(
            self.orifice_diameter, self.pipe_diameter, self.delta_pressure[0], self.delta_pressure[1], self.fluid_density, self.fluid_viscosity, self.expansion_factor)

    def volumetric_flow_rate_from_venturi_meter(self, units = 'm**3/s'):
        '''
        Sets self.volumetric_flow_rate to the result but doesn't return anything.
        '''
        self.has_values(['pipe_diameter', 'venturi_diameter', 'delta_pressure', 'fluid_density'])
        self.set_default_values(['expansion_factor'], [1])

        @ureg.wraps(units, ('m', 'm', 'kg/m**3', 'Pa', 'Pa', None, None))
        def volumetric_flow_rate_from_venturi_meter_function(
            venturi_diameter, pipe_diameter, fluid_density, initial_pressure, final_pressure, discharge_coefficient = 1, expansion_factor = 1):
            beta = venturi_diameter / pipe_diameter
            area = 3.1415 / 4 * venturi_diameter **2 # just a guess to get it started
            return expansion_factor * discharge_coefficient * area * (
                2 * (initial_pressure - final_pressure) / fluid_density / (1 - beta)**4)**0.5

        setattr(self, 'volumetric_flow_rate', volumetric_flow_rate_from_venturi_meter_function(
            self.venturi_diameter, self.pipe_diameter, self.fluid_density, self.delta_pressure[0],
            self.delta_pressure[1], self.discharge_coefficient, self.expansion_factor))
        return

# The above functions are finders. Some of them return things.
########################################################
# The below functions are residuals.

    def mechanical_energy_balance_residual(self, form_type):
        '''
        Form_type takes the arguments:
            'head'
            'pressure'
            'energy'
        Returns
        -------
        Residual of the mechanical energy balance equation. Values must be in SI units.
        '''
        self.has_values(['fluid_density'])
        self.set_default_values(
            ['initial_pressure:Pa', 'final_pressure:Pa', 'initial_velocity:m/s', 'final_velocity:m/s', 'initial_height:m', 'final_height', 'initial_alpha', 'final_alpha'],
            [0, 0, 0, 0, 0, 0, 1, 1])

        @ureg.wraps(None, (None, 'kg/m**3', 'Pa', 'Pa', 'm/s', 'm/s', 'm', 'm', None, None, 'm', 'm', 'Pa', 'Pa', 'J/kg', 'J/kg'))
        def mechanical_energy_balance_residual_function(
            form_type, fluid_density, initial_pressure = 0 * ureg('Pa'), final_pressure = 0 * ureg('Pa'), initial_velocity = 0 * ureg('m'),
            final_velocity = 0 * ureg('m/s'), initial_height = 0* ureg('m'), final_height = 0 * ureg('m'), initial_alpha = 1, final_alpha = 1,
            pump_head = 0 * ureg('m'), head_loss = 0 * ureg('m'),
            pump_pressure = 0 * ureg('Pa'), pressure_loss = 0 * ureg('Pa'),
            pump_work = 0 * ureg('J/kg'), e_loss = 0 * ureg('J/kg')):
            '''
            This function takes a billion arguments, but you only have to fill out the ones that you need.
            It will return the residual, so be sure to use another function with opt.fsolve to solve it.
            '''
            g = 9.8  # m/s2

            if form_type == 'head':
                pressure_term = (final_pressure - initial_pressure) / fluid_density / g
                velocity_term = 0.5 * (final_alpha * final_velocity**2
                                    - initial_alpha * initial_velocity**2) / g
                height_term = final_height - initial_height

                self.set_default_values(['pump_head', 'head_loss'], [0, 0])
                return pressure_term + velocity_term + height_term - pump_head + head_loss

            elif form_type == 'pressure':
                pressure_term = final_pressure - initial_pressure
                velocity_term = 0.5 * fluid_density * (final_alpha * final_velocity**2
                                                            - initial_alpha * initial_velocity**2)
                height_term = g * fluid_density * (final_height - initial_height)

                self.set_default_values(['pump_pressure', 'pressure_loss'], [0, 0])
                return pressure_term + velocity_term + height_term - pump_pressure + pressure_loss

            elif form_type == 'energy':
                pressure_term = (final_pressure - initial_pressure) / fluid_density
                velocity_term = 0.5 * (final_alpha * final_velocity**2 -
                                    initial_alpha * initial_velocity**2)
                height_term = g * (final_height - initial_height)

                self.set_default_values(['pump_work', 'e_loss'], [0, 0])
                return pressure_term + velocity_term + height_term - pump_work + e_loss

        if form_type == 'head':
            self.set_default_values(['pump_head:m', 'head_loss:m'], [0, 0])
            return mechanical_energy_balance_residual_function(
            form_type, self.fluid_density, self.initial_pressure, self.final_pressure, self.initial_velocity, self.final_velocity, self.initial_height, self.final_height,
            self.initial_alpha, self.final_alpha, self.pump_head, self.head_loss)

        elif form_type == 'pressure':
            self.set_default_values(['pump_head:m', 'head_loss:m'], [0, 0])
            return mechanical_energy_balance_residual_function(
            form_type, self.fluid_density, self.initial_pressure, self.final_pressure, self.initial_velocity, self.final_velocity, self.initial_height, self.final_height,
            self.initial_alpha, self.final_alpha, pump_pressure = self.pump_pressure, pressure_loss = self.pressure_loss)

        elif form_type == 'energy':
            self.set_default_values(['pump_head:m', 'head_loss:m'], [0, 0])
            return mechanical_energy_balance_residual_function(
            form_type, self.fluid_density, self.initial_pressure, self.final_pressure, self.initial_velocity, self.final_velocity, self.initial_height, self.final_height,
            self.initial_alpha, self.final_alpha, pump_work = self.pump_work, e_loss = self.e_loss)

# The above functions are residuals.
########################################################
# The below functions are probably not going to be used more than when I made them, but why not?

    def relief_molar_flow_finder(self):
        '''
        Uses 311 equation to find molar flow rate.
        '''
        self.has_values(['tank_pressure', 'tank_temperature', 'molecular_weight'])
        self.set_default_values(['k', 'discharge_coefficient', 'molar_flow_rate_units'], [1.4, 1, 'mol/s'])
        area = 3.1415 / 4 * self.orifice_diameter**2
        self.mach_number_relief_sizing()

        setattr(self, 'molar_flow_rate',
                self.tank_pressure * area * self.discharge_coefficient * (self.k / 8.314 / self.tank_temperature / self.molecular_weight)**0.5 * self.mach_number *
                (1 + (self.k - 1) / 2 * self.mach_number**2 ** ((self.k + 1) / (2 - 2 * self.k))))
        return

    def time_to_empty_tank(self, diameter = False):
        '''
        Used this for the BFR
        '''
        self.has_values(['tank_pressure', 'tank_temperature', 'tank_volume'])
        time = 0
        self.orifice_diameter = diameter if diameter else self.orifice_diameter
        initial_pressure = self.tank_pressure
        while self.tank_pressure > (10+12) * 6895: # assume pressure drop of 10 psi
            # calculate flow based on everything (including an assumed diameter)
            self.relief_molar_flow_finder()
            moles_after_flow = self.tank_pressure * self.tank_volume / 8.314 / self.tank_temperature - self.molar_flow_rate * 30
            self.tank_pressure = moles_after_flow * 8.314 * self.tank_temperature / self.tank_volume
            # print(f'{(self.tank_pressure / 6895 - 12):.2F},  {(self.molar_flow_rate * self.molecular_weight * 3600):.2F}')
            time += 30
            # wait 30 sec and calculate a new pressure based the flow
        self.tank_pressure = initial_pressure
        return time / 60

    def diameter_from_time_to_empty(self, desired_time):
        '''
        Used for the BFR.
        '''
        def residual(diameter):
            self.orifice_diameter = diameter[0] * 2.54 / 100
            return desired_time - self.time_to_empty_tank()
        print(f'{(desired_time / 480 * 100):.2F} %')
        return opt.fsolve(residual, 0.01)[0]


In [ ]:
large_tee_straight = 0.9
large_tee_bend = 2
small_tee_straight = 0.9
small_tee_bend = 2
half_inch_globe_valve = 10 # open

large_change_bend = 0.9
small_change_bend = 0.9
large_bend = 0.9
small_bend = 0.9

toilet = 14
sink = 10

# each pipe includes its section plus the fitting closer to the pump

basement_pipe = Link(System('pipe_diameter:in', 1,
                       'epsilon:mm', 0.0015,
                       'fluid_density:lb/ft**3', 62.3,
                       'fluid_viscosity:lb/ft/s', 6.733e-4,
                       'pipe_length:ft', 20,
                       'name', 'basement_pipe',
                       'loss_coefficients', [0]))
pipe1 = Link(System('pipe_diameter:in', 3/4,
                       'epsilon:mm', 0.0015,
                       'fluid_density:lb/ft**3', 62.3,
                       'fluid_viscosity:lb/ft/s', 6.733e-4,
                       'pipe_length:ft', 3,
                       'name', 'pipe1',
                       'loss_coefficients', [large_tee_bend]),
                       basement_pipe)
pipe2 = Link(System('pipe_diameter:in', 3/4,
                       'epsilon:mm', 0.0015,
                       'fluid_density:lb/ft**3', 62.3,
                       'fluid_viscosity:lb/ft/s', 6.733e-4,
                       'pipe_length:ft', 3,
                       'name', 'pipe2',
                       'loss_coefficients', [small_tee_straight]),
                       pipe1)
pipe3 = Link(System('pipe_diameter:in', 3/4,
                       'epsilon:mm', 0.0015,
                       'fluid_density:lb/ft**3', 62.3,
                       'fluid_viscosity:lb/ft/s', 6.733e-4,
                       'pipe_length:ft', 3,
                       'name', 'pipe3',
                       'loss_coefficients', [small_tee_straight]),
                       pipe2)
pipe4 = Link(System('pipe_diameter:in', 3/4,
                       'epsilon:mm', 0.0015,
                       'fluid_density:lb/ft**3', 62.3,
                       'fluid_viscosity:lb/ft/s', 6.733e-4,
                       'pipe_length:ft', 8.5,
                       'name', 'pipe4',
                       'loss_coefficients', [small_tee_straight, large_bend, large_bend]),
                       pipe3)
pipe5 = Link(System('pipe_diameter:in', 3/4,
                       'epsilon:mm', 0.0015,
                       'fluid_density:lb/ft**3', 62.3,
                       'fluid_viscosity:lb/ft/s', 6.733e-4,
                       'pipe_length:ft', 3,
                       'name', 'pipe5',
                       'loss_coefficients', [small_tee_straight]),
                       pipe4)
wall_pipe = Link(System('pipe_diameter:in', 1,
                       'epsilon:mm', 0.0015,
                       'fluid_density:lb/ft**3', 62.3,
                       'fluid_viscosity:lb/ft/s', 6.733e-4,
                       'pipe_length:ft', 20,
                       'name', 'wall_pipe',
                       'loss_coefficients', [large_tee_straight]),
                       basement_pipe)
floor2 = Link(System('pipe_diameter:in', 3/4,
                       'epsilon:mm', 0.0015,
                       'fluid_density:lb/ft**3', 62.3,
                       'fluid_viscosity:lb/ft/s', 6.733e-4,
                       'pipe_length:ft', 3,
                       'name', 'floor2',
                       'loss_coefficients', [large_change_bend]),
                       wall_pipe)
pipe6 = Link(System('pipe_diameter:in', 3/4,
                       'epsilon:mm', 0.0015,
                       'fluid_density:lb/ft**3', 62.3,
                       'fluid_viscosity:lb/ft/s', 6.733e-4,
                       'pipe_length:ft', 3,
                       'name', 'pipe6',
                       'loss_coefficients', [small_tee_straight]),
                       floor2)
pipe7 = Link(System('pipe_diameter:in', 3/4,
                       'epsilon:mm', 0.0015,
                       'fluid_density:lb/ft**3', 62.3,
                       'fluid_viscosity:lb/ft/s', 6.733e-4,
                       'pipe_length:ft', 3,
                       'name', 'pipe7',
                       'loss_coefficients', [small_tee_straight]),
                       pipe6)
pipe8 = Link(System('pipe_diameter:in', 3/4,
                       'epsilon:mm', 0.0015,
                       'fluid_density:lb/ft**3', 62.3,
                       'fluid_viscosity:lb/ft/s', 6.733e-4,
                       'pipe_length:ft', 8.5,
                       'name', 'pipe8',
                       'loss_coefficients', [small_tee_straight, large_bend, large_bend]),
                       pipe7)
pipe9 = Link(System('pipe_diameter:in', 3/4,
                       'epsilon:mm', 0.0015,
                       'fluid_density:lb/ft**3', 62.3,
                       'fluid_viscosity:lb/ft/s', 6.733e-4,
                       'pipe_length:ft', 3,
                       'name', 'pipe9',
                       'loss_coefficients', [small_tee_straight]),
                       pipe8)
toilet_1_1 = Link(System('pipe_diameter:in', 1/2,
                       'epsilon:mm', 0.0015,
                       'fluid_density:lb/ft**3', 62.3,
                       'fluid_viscosity:lb/ft/s', 6.733e-4,
                       'pipe_length:ft', 4,
                       'name', 'toilet_1_1',
                       'loss_coefficients', [small_tee_bend, small_bend, half_inch_globe_valve, toilet]),
                       pipe1)
toilet_1_2 = Link(System('pipe_diameter:in', 1/2,
                       'epsilon:mm', 0.0015,
                       'fluid_density:lb/ft**3', 62.3,
                       'fluid_viscosity:lb/ft/s', 6.733e-4,
                       'pipe_length:ft', 4,
                       'name', 'toilet_1_2',
                       'loss_coefficients', [small_tee_bend, small_bend, half_inch_globe_valve, toilet]),
                       pipe2)
toilet_1_3 = Link(System('pipe_diameter:in', 1/2,
                       'epsilon:mm', 0.0015,
                       'fluid_density:lb/ft**3', 62.3,
                       'fluid_viscosity:lb/ft/s', 6.733e-4,
                       'pipe_length:ft', 4,
                       'name', 'toilet_1_3',
                       'loss_coefficients', [small_tee_bend, small_bend, half_inch_globe_valve, toilet]),
                       pipe3)
toilet_2_1 = Link(System('pipe_diameter:in', 1/2,
                       'epsilon:mm', 0.0015,
                       'fluid_density:lb/ft**3', 62.3,
                       'fluid_viscosity:lb/ft/s', 6.733e-4,
                       'pipe_length:ft', 4,
                       'name', 'toilet_2_1',
                       'loss_coefficients', [small_tee_bend, small_bend, half_inch_globe_valve, toilet]),
                       floor2)
toilet_2_2 = Link(System('pipe_diameter:in', 1/2,
                       'epsilon:mm', 0.0015,
                       'fluid_density:lb/ft**3', 62.3,
                       'fluid_viscosity:lb/ft/s', 6.733e-4,
                       'pipe_length:ft', 4,
                       'name', 'toilet_2_2',
                       'loss_coefficients', [small_tee_bend, small_bend, half_inch_globe_valve, toilet]),
                       pipe6)
toilet_2_3 = Link(System('pipe_diameter:in', 1/2,
                       'epsilon:mm', 0.0015,
                       'fluid_density:lb/ft**3', 62.3,
                       'fluid_viscosity:lb/ft/s', 6.733e-4,
                       'pipe_length:ft', 4,
                       'name', 'toilet_2_3',
                       'loss_coefficients', [small_tee_bend, small_bend, half_inch_globe_valve, toilet]),
                       pipe7)
sink_1_1 = Link(System('pipe_diameter:in', 1/2,
                       'epsilon:mm', 0.0015,
                       'fluid_density:lb/ft**3', 62.3,
                       'fluid_viscosity:lb/ft/s', 6.733e-4,
                       'name', 'sink_1_1',
                       'pipe_length:ft', 4,
                       'loss_coefficients', [small_tee_bend, small_bend, half_inch_globe_valve, sink]),
                       pipe4)
sink_1_2 = Link(System('pipe_diameter:in', 1/2,
                       'epsilon:mm', 0.0015,
                       'fluid_density:lb/ft**3', 62.3,
                       'fluid_viscosity:lb/ft/s', 6.733e-4,
                       'pipe_length:ft', 4,
                       'name', 'sink_1_2',
                       'loss_coefficients', [small_tee_bend, small_bend, half_inch_globe_valve, sink]),
                       pipe5)
pipe11 = Link(System('pipe_diameter:in', 3/4,
                       'epsilon:mm', 0.0015,
                       'fluid_density:lb/ft**3', 62.3,
                       'fluid_viscosity:lb/ft/s', 6.733e-4,
                       'pipe_length:ft', 3,
                       'name', 'pipe10',
                       'loss_coefficients', [small_tee_straight]),
                       pipe5)
sink_1_3 = Link(System('pipe_diameter:in', 1/2,
                       'epsilon:mm', 0.0015,
                       'fluid_density:lb/ft**3', 62.3,
                       'fluid_viscosity:lb/ft/s', 6.733e-4,
                       'pipe_length:ft', 4,
                       'name', 'sink_1_3',
                       'loss_coefficients', [small_bend, small_change_bend, half_inch_globe_valve, sink]),
                       pipe11)
sink_2_1 = Link(System('pipe_diameter:in', 1/2,
                       'epsilon:mm', 0.0015,
                       'fluid_density:lb/ft**3', 62.3,
                       'fluid_viscosity:lb/ft/s', 6.733e-4,
                       'pipe_length:ft', 4,
                       'name', 'sink_2_1',
                       'loss_coefficients', [small_tee_bend, small_bend, half_inch_globe_valve, sink]),
                       pipe8)
sink_2_2 = Link(System('pipe_diameter:in', 1/2,
                       'epsilon:mm', 0.0015,
                       'fluid_density:lb/ft**3', 62.3,
                       'fluid_viscosity:lb/ft/s', 6.733e-4,
                       'pipe_length:ft', 4,
                       'name', 'sink_2_2',
                       'loss_coefficients', [small_tee_bend, small_bend, half_inch_globe_valve, sink]),
                       pipe9)
pipe10 = Link(System('pipe_diameter:in', 3/4,
                       'epsilon:mm', 0.0015,
                       'fluid_density:lb/ft**3', 62.3,
                       'fluid_viscosity:lb/ft/s', 6.733e-4,
                       'pipe_length:ft', 3,
                       'name', 'pipe10',
                       'loss_coefficients', [small_tee_straight]),
                       pipe9)
sink_2_3 = Link(System('pipe_diameter:in', 1/2,
                       'epsilon:mm', 0.0015,
                       'fluid_density:lb/ft**3', 62.3,
                       'fluid_viscosity:lb/ft/s', 6.733e-4,
                       'pipe_length:ft', 4,
                       'volumetric_flow_rate:gal/min', 1.34,
                       'name', 'sink_2_3',
                       'loss_coefficients', [small_bend, small_change_bend, half_inch_globe_valve, sink]),
                       pipe10)
all_appliances = [toilet_1_1, toilet_1_2, toilet_1_3, sink_1_1, sink_1_2, sink_1_3, toilet_2_1, toilet_2_2, toilet_2_3, sink_2_1, sink_2_2, sink_2_3]
pipes = [basement_pipe, wall_pipe, floor2, pipe1, pipe2, pipe3, pipe4, pipe5, pipe6, pipe7, pipe8, pipe9, pipe10, pipe11]



In [ ]:
def find_head_loss(section):
    '''
    Assumes known volumetric flow rate.
    '''
    section.average_velocity_finder()
    section.reynolds_number_finder()
    section.friction_factor_using_colebrook_finder()
    section.head_loss_finder()
    return

def total_head_loss(appliance):

    link = appliance
    total = 0 * ureg.ft

    while link is not Link.empty:

        find_head_loss(link.first)
        total += link.first.head_loss
        link = link.rest

    # print(appliance.first.name, 'head loss = ', total) # debug

    return total

def make_losses_equal(known: Link, unknown: Link):

    # print(unknown.rest.first.name) # debug
    # link = known # debug
    # while 34: # debug
    #     if link is unknown.rest: # debug
    #         break # debug
    #     link = link.rest # debug
    #     print('  link', link.first.name) # debug

    def residual(flow_rate):
        '''
        add the total from the known down to where it is one past the unknown

        '''
        setattr(unknown.first, 'volumetric_flow_rate', flow_rate[0] * ureg('gal/min'))
        # setattr(unknown.rest.first, 'volumetric_flow_rate', unknown.first.volumetric_flow_rate + known.first.volumetric_flow_rate)
        find_head_loss(unknown.first)

        link = known
        total_known_head_loss = 0 * ureg.ft

        while 34:

            find_head_loss(link.first)
            total_known_head_loss += link.first.head_loss

            if link.rest is unknown.rest:
                setattr(unknown.rest.first, 'volumetric_flow_rate', unknown.first.volumetric_flow_rate + link.first.volumetric_flow_rate)
                # print(f'unknown is {unknown.first.name}. Link is {link.first.name}') # debug

            link = link.rest
            if link is unknown.rest:
                break

        find_head_loss(known.rest.first)
        return (unknown.first.head_loss - total_known_head_loss).magnitude

    # print(f'{unknown.rest.first.name} flow rate was set to {unknown.rest.first.volumetric_flow_rate}.') # debug
    return opt.fsolve(residual, 1)

def find_all():
    find_head_loss(sink_2_3.first)
    setattr(pipe10.first, 'volumetric_flow_rate', sink_2_3.first.volumetric_flow_rate)
    make_losses_equal(sink_2_3, sink_2_2)
    make_losses_equal(sink_2_3, sink_2_1)
    make_losses_equal(sink_2_3, toilet_2_3)
    make_losses_equal(sink_2_3, toilet_2_2)
    make_losses_equal(sink_2_3, toilet_2_1)
    setattr(wall_pipe.first, 'volumetric_flow_rate', floor2.first.volumetric_flow_rate)
    find_head_loss(wall_pipe.first)
    # setattr(wall_pipe.first, 'head_loss', floor2.first.head_loss + wall_pipe.first.head_loss)

    def find_first_floor(flow_rate):
        setattr(sink_1_3.first, 'volumetric_flow_rate', flow_rate)
        setattr(pipe11.first, 'volumetric_flow_rate', sink_1_3.first.volumetric_flow_rate)
        find_head_loss(sink_1_3.first)
        make_losses_equal(sink_1_3, sink_1_2)
        make_losses_equal(sink_1_3, sink_1_1)
        make_losses_equal(sink_1_3, toilet_1_3)
        make_losses_equal(sink_1_3, toilet_1_2)
        make_losses_equal(sink_1_3, toilet_1_1)

        setattr(basement_pipe.first, 'volumetric_flow_rate', pipe1.first.volumetric_flow_rate + wall_pipe.first.volumetric_flow_rate)
        find_head_loss(basement_pipe.first)

        # print(total_head_loss(sink_2_3), sink_2_3.first.volumetric_flow_rate, sink_2_2.first.volumetric_flow_rate) # debug
        return

    # find_first_floor( * ureg('gal/min')) # debug
    def residual(flow_rate):
        find_first_floor(flow_rate[0] * ureg('gal/min'))
        return (total_head_loss(sink_2_3) + 20 * ureg.ft - total_head_loss(toilet_1_1)).magnitude

    opt.fsolve(residual, 1)

    return

find_all()


In [ ]:
print(f'The total volumetric_flow_rate is {basement_pipe.first.volumetric_flow_rate}.\n')

for i in all_appliances + pipes:
    print(i.first.name, '   ', total_head_loss(i), '    ',i.first.volumetric_flow_rate)

The total volumetric_flow_rate is 28.23628930521508 gallon / minute.

toilet_1_1     36.355147204950114 foot      3.875142288144268 gallon / minute
toilet_1_2     36.355147204950114 foot      3.472435440893571 gallon / minute
toilet_1_3     36.355147204950114 foot      3.200548074251422 gallon / minute
sink_1_1     36.35514720495007 foot      2.9166259846756923 gallon / minute
sink_1_2     36.3551472049501 foot      2.823862436163831 gallon / minute
sink_1_3     36.355147204950114 foot      2.8605796704675246 gallon / minute
toilet_2_1     16.35514720495012 foot      1.8636825315681942 gallon / minute
toilet_2_2     16.35514720495012 foot      1.6615614677323631 gallon / minute
toilet_2_3     16.35514720495009 foot      1.5246564162684098 gallon / minute
sink_2_1     16.35514720495012 foot      1.3724198658062627 gallon / minute
sink_2_2     16.35514720495012 foot      1.3247751292435441 gallon / minute
sink_2_3     16.35514720495012 foot      1.34 gallon / minute
basement_pipe     9.3

In [ ]:
def pipe_economics():
    price = 0
    prices = {half_inch_globe_valve:50,
              large_tee_straight:0.60,
              small_tee_straight:0.30,
              large_change_bend:0.60,
              small_change_bend:0.30,
              large_bend:0.30,
              small_bend:0.20,
              }
    for i in all_appliances + pipes:
        for k in i.first.loss_coefficients:
            if k in prices:
                price += prices[k]

        if i.first.pipe_diameter == 0.5 * ureg('in'):
            price += i.first.pipe_length.to('ft').magnitude * 0.75

        if i.first.pipe_diameter == 0.75 * ureg('in'):
            price += i.first.pipe_length.to('ft').magnitude * 1

        if i.first.pipe_diameter == 1 * ureg('in'):
            price += i.first.pipe_length.to('ft').magnitude * 1.25

    return '$' + str(round(price, 2))

print(pipe_economics())

test = System('pipe_diameter:in', 1/2,
              'volumetric_flow_rate:gal/min', 12,
              'epsilon:mm', 0.0015,
              'fluid_density:lb/ft**3', 62.3,
              'fluid_viscosity:lb/ft/s', 6.733e-4,
              'pipe_length:ft', 20,
            #   'loss_coefficients',
              )


$1039.0
